In [97]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

In [98]:
property_sales = pd.read_csv("NYC_Property_Sales.csv")
air_quality_cleaned = pd.read_csv("air_quality_cleaned.csv")
crime_cleaned = pd.read_csv("crime_cleaned.csv")

In [99]:
print(property_sales)

       Unnamed: 0  BOROUGH   NEIGHBORHOOD  \
0               4        1  ALPHABET CITY   
1               5        1  ALPHABET CITY   
2               6        1  ALPHABET CITY   
3               7        1  ALPHABET CITY   
4               8        1  ALPHABET CITY   
...           ...      ...            ...   
84543        8409        5        WOODROW   
84544        8410        5        WOODROW   
84545        8411        5        WOODROW   
84546        8412        5        WOODROW   
84547        8413        5        WOODROW   

                           BUILDING CLASS CATEGORY TAX CLASS AT PRESENT  \
0      07 RENTALS - WALKUP APARTMENTS                                2A   
1      07 RENTALS - WALKUP APARTMENTS                                 2   
2      07 RENTALS - WALKUP APARTMENTS                                 2   
3      07 RENTALS - WALKUP APARTMENTS                                2B   
4      07 RENTALS - WALKUP APARTMENTS                                2A   
...      

In [100]:
#Clean missing values
property_sales_cleaned = property_sales.drop(columns=['EASE-MENT'])
property_sales_cleaned = property_sales_cleaned.replace('-', np.NaN)
property_sales_cleaned = property_sales_cleaned.replace(r'^\s*$', np.NaN, regex=True)
property_sales_cleaned = property_sales_cleaned.fillna(0)

In [101]:
print(property_sales_cleaned['NEIGHBORHOOD'].unique().size)
print(property_sales_cleaned['BUILDING CLASS CATEGORY'].unique().size)
print(property_sales_cleaned['TAX CLASS AT PRESENT'].unique().size)
print(property_sales_cleaned['BUILDING CLASS AT PRESENT'].unique().size)
print(property_sales_cleaned['ADDRESS'].unique().size)
print(property_sales_cleaned['TAX CLASS AT TIME OF SALE'].unique().size)
print(property_sales_cleaned['BUILDING CLASS AT TIME OF SALE'].unique().size)

254
47
11
167
67563
4
166


In [102]:
#Convert SALE DATE from datetime to two new columns SALE MONTH and SALE YEAR
property_sales_cleaned['SALE DATE'] = pd.to_datetime(property_sales_cleaned['SALE DATE'])
property_sales_cleaned['SALE MONTH'] = property_sales_cleaned['SALE DATE'].dt.month
property_sales_cleaned['SALE YEAR'] = property_sales_cleaned['SALE DATE'].dt.year
property_sales_cleaned.drop('SALE DATE', axis=1, inplace=True)

In [103]:
#Convert codes in property sales to borough names
borough_codes = {1: 'Manhattan', 2: 'Bronx', 3: 'Brooklyn', 4: 'Queens', 5: 'Staten Island'}
property_sales_cleaned['borough'] = property_sales_cleaned['BOROUGH'].apply(lambda x: borough_codes[x])
property_sales_cleaned.drop('BOROUGH', axis=1, inplace=True)

#Merge cleaned dataframes
property_sales_cleaned = property_sales_cleaned.merge(crime_cleaned, on='borough', how='left')
property_sales_cleaned = property_sales_cleaned.merge(air_quality_cleaned, on='borough', how='left')

#Rename columns to keep naming scheme
property_sales_cleaned.rename(columns={
    'violent_crime_count': 'VIOLENT CRIME COUNT', 'nonviolent_crime_count': 'NONVIOLENT CRIMECOUNT',
    'average_pm2.5': 'AVERAGE PM2.5', 'borough':'BOROUGH'
}, inplace=True)

In [104]:
#One-hot encode the categorical columns except address
property_sales_cleaned = pd.get_dummies(property_sales_cleaned, columns = [
    'NEIGHBORHOOD', 'BUILDING CLASS CATEGORY', 'TAX CLASS AT PRESENT',
    'BUILDING CLASS AT PRESENT', 'TAX CLASS AT TIME OF SALE','BUILDING CLASS AT TIME OF SALE',
    'BOROUGH'
], dtype=float)
print(property_sales_cleaned)

       Unnamed: 0_x  BLOCK  LOT                 ADDRESS APARTMENT NUMBER  \
0                 4    392    6            153 AVENUE B                0   
1                 5    399   26   234 EAST 4TH   STREET                0   
2                 6    399   39   197 EAST 3RD   STREET                0   
3                 7    402   21     154 EAST 7TH STREET                0   
4                 8    404   55  301 EAST 10TH   STREET                0   
...             ...    ...  ...                     ...              ...   
84543          8409   7349   34           37 QUAIL LANE                0   
84544          8410   7349   78        32 PHEASANT LANE                0   
84545          8411   7351   60        49 PITNEY AVENUE                0   
84546          8412   7100   28   2730 ARTHUR KILL ROAD                0   
84547          8413   7105  679       155 CLAY PIT ROAD                0   

       ZIP CODE  RESIDENTIAL UNITS  COMMERCIAL UNITS  TOTAL UNITS  \
0         10009   

In [105]:
#Label-encode address
label_encoder = preprocessing.LabelEncoder()
property_sales_cleaned['ADDRESS'] = label_encoder.fit_transform(property_sales_cleaned['ADDRESS'])

In [106]:
print(property_sales_cleaned)

       Unnamed: 0_x  BLOCK  LOT  ADDRESS APARTMENT NUMBER  ZIP CODE  \
0                 4    392    6    14889                0     10009   
1                 5    399   26    28592                0     10009   
2                 6    399   39    22192                0     10009   
3                 7    402   21    15090                0     10009   
4                 8    404   55    35431                0     10009   
...             ...    ...  ...      ...              ...       ...   
84543          8409   7349   34    41385                0     10309   
84544          8410   7349   78    37214                0     10309   
84545          8411   7351   60    48075                0     10309   
84546          8412   7100   28    33114                0     10309   
84547          8413   7105  679    15264                0     10309   

       RESIDENTIAL UNITS  COMMERCIAL UNITS  TOTAL UNITS LAND SQUARE FEET  ...  \
0                      5                 0            5           

In [107]:
property_sales_cleaned.to_csv("property_sales_cleaned.csv")